<a href="https://colab.research.google.com/github/eyet7/Markowitz-portfolio-optimization/blob/main/notebooks/01_markowitz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Core libraries for data handling, visualization, and financial data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
plt.style.use("seaborn-v0_8")

In [2]:
# Assests selection
tickers = ["AAPL", "MSFT","GOOGL", "AMZN", "SPY"]

#Download historical adjusted prices
prices = yf.download(
    tickers,
    start='2019-01-01',
    end='2024-01-01',
    auto_adjust=True
)
prices.head()


[*********************100%***********************]  5 of 5 completed


Price           Close                                                    High  \
Ticker           AAPL       AMZN      GOOGL       MSFT         SPY       AAPL   
Date                                                                            
2019-01-02  37.538815  76.956497  52.338543  94.612617  224.995331  37.759885   
2019-01-03  33.799671  75.014000  50.888996  91.132004  219.626312  34.638779   
2019-01-04  35.242558  78.769501  53.499275  95.370476  226.982864  35.311495   
2019-01-07  35.164116  81.475502  53.392586  95.492126  228.772522  35.378055   
2019-01-08  35.834461  82.829002  53.861538  96.184486  230.921967  36.088810   

Price                                                    ...       Open  \
Ticker           AMZN      GOOGL       MSFT         SPY  ...       AAPL   
Date                                                     ...              
2019-01-02  77.667999  52.641752  95.202072  225.921657  ...  36.818560   
2019-01-03  76.900002  52.913203  93.742459  223.547408  ...  34.225167   
2019-01-04  79.699997  53.595052  95.913153  227.630386  ...  34.355908   
2019-01-07  81.727997  53.729041  96.624258  230.184469  ...  35.347152   
2019-01-08  83.830498  54.257544  97.279191  231.407615  ...  35.551588   

Price                                                       Volume             \
Ticker           AMZN      GOOGL       MSFT         SPY       AAPL       AMZN   
Date                                                                            
2019-01-02  73.260002  50.974846  93.143652  221.218131  148158800  159662000   
2019-01-03  76.000504  52.139549  93.658247  223.241624  365248800  139512000   
2019-01-04  76.500000  51.737089  93.302697  222.666059  234428400  183652000   
2019-01-07  80.115501  53.643191  95.099156  227.252646  219111200  159864000   
2019-01-08  83.234497  53.892800  96.409040  230.966950  164101200  177628000   

Price                                      
Ticker         GOOGL      MSFT        SPY  
Date                                       
2019-01-02  31868000  35329300  126925200  
2019-01-03  41960000  42579100  144140700  
2019-01-04  46022000  44060600  142628800  
2019-01-07  47446000  35656100  103139100  
2019-01-08  35414000  31514400  102512600  

[5 rows x 25 columns]

In [3]:
prices.shape

(1258, 25)

In [4]:
# Keep only adjusted close prices.
prices_close = prices["Close"]
prices_close.head()

Ticker,AAPL,AMZN,GOOGL,MSFT,SPY
Date,,,,,
2019-01-02,37.538815,76.956497,52.338543,94.612617,224.995331
2019-01-03,33.799671,75.014000,50.888996,91.132004,219.626312
2019-01-04,35.242558,78.769501,53.499275,95.370476,226.982864
2019-01-07,35.164116,81.475502,53.392586,95.492126,228.772522
2019-01-08,35.834461,82.829002,53.861538,96.184486,230.921967


In [5]:
# Compute daily returns
returns = prices_close.pct_change().dropna()
returns.head()

Ticker,AAPL,AMZN,GOOGL,MSFT,SPY
Date,,,,,
2019-01-03,-0.099607,-0.025241,-0.027696,-0.036788,-0.023863
2019-01-04,0.042689,0.050064,0.051294,0.046509,0.033496
2019-01-07,-0.002226,0.034353,-0.001994,0.001276,0.007885
2019-01-08,0.019063,0.016612,0.008783,0.007250,0.009396
2019-01-09,0.016982,0.001714,-0.003427,0.014300,0.004673


In [6]:
# Annualized expected returns
mu = returns.mean(axis=0) * 252

# Annalized covariance matrix
cov = returns.cov() * 252

mu, cov

(Ticker
 AAPL     0.378031
 AMZN     0.198360
 GOOGL    0.245988
 MSFT     0.320295
 SPY      0.167224
 dtype: float64,
 Ticker      AAPL      AMZN     GOOGL      MSFT       SPY
 Ticker                                                  
 AAPL    0.103907  0.070302  0.069207  0.074562  0.054450
 AMZN    0.070302  0.124055  0.073902  0.073346  0.047956
 GOOGL   0.069207  0.073902  0.101188  0.073616  0.050897
 MSFT    0.074562  0.073346  0.073616  0.092969  0.052788
 SPY     0.054450  0.047956  0.050897  0.052788  0.044082)

In [8]:
# Equal-weight portfolio

n_assets = len (mu)
w_equal = np.ones(n_assets) / n_assets
portfolio_return = np.dot(w_equal, mu)
portfolio_variance = np.dot(w_equal.T, np.dot(cov, w_equal))
portfolio_volatility = np.sqrt(portfolio_variance)

print(f"Expected annual return: {portfolio_return:.2%}")
print(f"Annual volatility: {portfolio_volatility:.2%}")


Expected annual return: 26.20%
Annual volatility: 26.44%


In [11]:
sharpe = portfolio_return / portfolio_volatility
print(f"Sharpe ratio (no risk-free rate): {sharpe:.2f}")


Sharpe ratio (no risk-free rate): 0.99


In [12]:
# Reproducibility
np.random.seed(42)

n_portfolios = 5000
n_assests = len (mu)

#Arrays to store results
port_returns = np.zeros(n_portfolios)
port_vols = np.zeros(n_portfolios)
port_sharpes = np.zeros(n_portfolios)
weights_store = np.zeros((n_portfolios, n_assests))

for i in range(n_portfolios):
  # Random long-only weights that sum to 1
  w = np.random.random(n_assets)
  w = w / w.sum()

  weights_store[i, :] = w

  # Portfolio return and risk
  r = np.dot(w, mu)
  v = np.sqrt(np.dot(w.T, np.dot(cov, w)))

  port_returns[i] = r
  port_vols[i] = v
  port_sharpes[i] = r / v


# Identify best Sharpe and minimum volatility portfolios
idx_max_sharpe = np.argmax(port_sharpes)
idx_min_vol = np.argmin(port_vols)

best_point = weights_store[idx_max_sharpe]
minvol_point = (port_vols[idx_min_vol], port_returns[idx_min_vol])

best_point, minvol_point

(array([0.54935411, 0.01615279, 0.09633147, 0.30528063, 0.032881  ]),
 (np.float64(0.2269803079295987), np.float64(0.21282133560227534)))